In [79]:
# Remove all mentions of blouse_DF and sunderas before writing to CSV, directly after making dataframe
# Another approach is to remove the JSON files of everything not MORIA.... and the MIROA with blouse_DF etc.
import json
import pandas as pd
from pandas.io.json import json_normalize 
from pandasql import sqldf
import os
from os import listdir
from os.path import isfile, join
DATAPATH = "../../data/Datasets_Enhancy/Annotation/"
IMAGEPATH = "../../data/Datasets_Enhancy/datasets/"


onlyfiles = [f for f in listdir(DATAPATH) if isfile(join(DATAPATH, f))]
ret = []
for f in onlyfiles:
    if ("MIROA") in f and not ("blouse" in f or "sudaderas" in f or "top_DF" in f):
        ret.append(f)
print(ret)
frames =[]
for i in range(len(ret)):
    with open(DATAPATH + ret[i]) as f:
        d = json.load(f)
        frames.append(pd.json_normalize(d['annotations']))
        
df = pd.concat(frames)
exportDf = df[['image_id', 'segmentation']]
#exportDf = exportDf[exportDf["image_id"].str.contains("blouse_DF") == False]
#exportDf = exportDf[exportDf["image_id"].str.contains("top_DF") == False]
#exportDf = exportDf[exportDf["image_id"].str.contains("sudaderas") == False]



deleted = 0
changed = 0
s = len(exportDf.index)
outPut = exportDf.copy()

for i in range(s):
    try:
        id = exportDf.image_id.iloc[i]
    except:
        print("BROKEN AT:id {} idx".format(id, i))
    path1 = IMAGEPATH + id
    path2 = IMAGEPATH + "dataset/" + id
    
    if not os.path.isfile(path1):
        if not os.path.isfile(path2):
            try:
                idx = outPut.loc[outPut['image_id'] == id].index[0]
                print("DROPPED AT idx {} path {}".format(i, path2))
                outPut = outPut.drop(idx, axis = 0)
                deleted += 1
            except:
                print("ALREADY DROPPED {}".format(path2))
            
        else:
            outPut.at[i, "image_id"] = path2
            changed += 1


print("Total: {}".format(s))    
print("Deleted: {}".format(deleted))
print("Edited: {}".format(changed))
outPut.to_csv(r'./export_data.csv')


#Not all images exists check this
# Different folder paths fix
# 


['MIROA_Hauts_Anorak_Bomber_DF_Complet.json', 'MIROA_Full_body_petos_complet.json', 'MIROA_Full_body_monos_complet.json', 'MIROA_Bas_jeans_complet_retravailJBN.json', 'MIROA_Hauts_Blazer_DF_Complet.json', 'MIROA_Full_body_vestidos_complet.json', 'MIROA_Hauts_cardigans_complet.json', 'MIROA_Bas_faldas_complet.json', 'MIROA_Hauts_camisas_y_blusas_complet.json', 'MIROA_Hauts_jerseys_complet.json', 'MIROA_Full_body_jacket_DF_complet.json', 'MIROA_Full_Body_Romper_DF_Complet.json', 'MIROA_Full_body_abrigos_complet.json', 'MIROA_Full_body_coat_DF_complet.json', 'MIROA_Full_Body_jumpsuit_DF_Complet.json']
DROPPED AT idx 6816 path ../../data/Datasets_Enhancy/datasets/dataset/blazer_DF/Boxy_Buttoned_Blazer/img_00000052.jpg
DROPPED AT idx 6817 path ../../data/Datasets_Enhancy/datasets/dataset/blazer_DF/Boxy_Buttoned_Blazer/img_00000052.jpg
DROPPED AT idx 6826 path ../../data/Datasets_Enhancy/datasets/dataset/blazer_DF/Boxy_Buttoned_Blazer/img_00000071.jpg
DROPPED AT idx 6849 path ../../data/Data

Total: 54461
Deleted: 5
Edited: 30532
